In [6]:
import numpy as np
from math import exp, sqrt
import matplotlib.pyplot as plt
import probDE.Bayesian as bo
#from probDE.Tests.root_gen import root_gen
from probDE.utils.utils import mvncond
from probDE.Kalman.kalman_initial_draw import kalman_initial_draw
from probDE.Kalman.kalman_ode_higher import kalman_ode_higher
from probDE.Kalman.higher_mvncond import higher_mvncond
from probDE.Kalman.cov_car import cov_car
from probDE.Kalman.Old.kalman_filter import kalman_filter
from probDE.Kalman.Old.kalman_smooth import kalman_smooth
from filter_update_full import filter_update_full
from pykalman import standard as pks
import scipy as sc

In [6]:
from math import sin, cos
def chk_F(y_t, t):
    return sin(2*t) - y_t[0] #X^{2} = sin(2t) - X

def chk_exact_x(t):
    return (-3*cos(t) + 2*sin(t) - sin(2*t))/3

def chk_exact_x1(t):
    return (-2*cos(2*t) + 3*sin(t) + 2*cos(t))/3

def chk_exact_x2(t):
    return (4*sin(2*t) - 2*sin(t) + 3*cos(t))/3


# Test Against Pykalman

In [7]:
#Define parameters
N = 2
q = 2
p = q+2

delta_t = np.array([1/N])
r0 = 0.5
sigma = 0.001
roots = root_gen(r0, p) #Generate roots to draw x^{(3)}_0
a = np.array([0,0,1])
x0 = np.array([-1,0,0])
x0 = np.array([-1,0,chk_F(x0, 0)]) #Initial state

Y0 = kalman_initial_draw(roots, sigma, x0, p)
A, V = higher_mvCond(delta_t, roots, sigma) 
lam = np.zeros(p)
b = lam - A.dot(lam)
fun = chk_F

In [7]:
import numpy as np
from pykalman import standard as pks
from kalmantv.cython import KalmanTV

In [9]:
def rel_err(X1, X2):
    """Relative error between two numpy arrays."""
    return np.max(np.abs((X1.ravel() - X2.ravel())/X1.ravel()))


def rand_vec(n):
    """Generate a random vector."""
    return np.random.randn(n)


def rand_mat(n, p=None, pd=True):
    """Generate a random matrix, positive definite if `pd = True`."""
    if p is None:
        p = n
    V = np.zeros((n, p), order='F')
    V[:] = np.random.randn(n, p)
    if (p == n) & pd:
        V[:] = np.matmul(V, V.T)
    return V

In [30]:
def test_predict():
    n_meas = np.random.randint(5)
    n_state = n_meas + np.random.randint(5)
    mu_state_past = rand_vec(n_state)
    var_state_past = rand_mat(n_state)
    mu_state = rand_vec(n_state)
    wgt_state = rand_mat(n_state, pd=False)
    var_state = rand_mat(n_state)
    mu_state_pred, var_state_pred = (
        pks._filter_predict(
            wgt_state, var_state,
            mu_state, mu_state_past,
            var_state_past
            )
    )
    ktv = KalmanTV(n_meas, n_state)
    mu_state_pred2 = np.empty(n_state)
    var_state_pred2 = np.empty((n_state, n_state), order='F')
    ktv.predict(mu_state_pred2, var_state_pred2,
                mu_state_past, var_state_past,
                mu_state, wgt_state, var_state)
    if np.isclose(rel_err(mu_state_pred, mu_state_pred2), 0.0) and \
        np.isclose(rel_err(var_state_pred, var_state_pred2), 0.0):
            return True
    return False

In [35]:
def test_update():
    n_meas = np.random.randint(5) + 1
    n_state = n_meas + np.random.randint(5)
    mu_state_pred = rand_vec(n_state)
    var_state_pred = rand_mat(n_state)
    x_meas = rand_vec(n_meas)
    mu_meas = rand_vec(n_meas)
    wgt_meas = rand_mat(n_meas, n_state, pd=False)
    var_meas = rand_mat(n_meas)
    _, mu_state_filt, var_state_filt = (
        pks._filter_correct(
            wgt_meas, var_meas,
            mu_meas, mu_state_pred,
            var_state_pred, x_meas
            )
        )
    ktv = KalmanTV(n_meas, n_state)
    mu_state_filt2 = np.empty(n_state)
    var_state_filt2 = np.empty((n_state, n_state), order='F')
    ktv.update(mu_state_filt2, var_state_filt2,
               mu_state_pred, var_state_pred,
               x_meas, mu_meas, wgt_meas, var_meas)
    if np.isclose(rel_err(mu_state_filt, mu_state_filt2), 0.0) and \
        np.isclose(rel_err(var_state_filt, var_state_filt2), 0.0):
            return True
    return False

In [47]:
def test_smooth_mv():
    n_meas = np.random.randint(5) + 2
    n_state = n_meas + np.random.randint(5)
    mu_state_next = rand_vec(n_state)
    var_state_next = rand_mat(n_state)
    mu_state_filt = rand_vec(n_state)
    var_state_filt = rand_mat(n_state)
    mu_state_pred = rand_vec(n_state)
    var_state_pred = rand_mat(n_state)
    wgt_state = rand_mat(n_state, pd=False)
    mu_state_smooth, var_state_smooth, _ = (
        pks._smooth_update(
            wgt_state, mu_state_filt,
            var_state_filt, mu_state_pred,
            var_state_pred, mu_state_next,
            var_state_next)
    )
    ktv = KalmanTV(n_meas, n_state)
    mu_state_smooth2 = np.empty(n_state)
    var_state_smooth2 = np.empty((n_state, n_state), order='F')
    ktv.smooth_mv(mu_state_smooth2, var_state_smooth2,
                  mu_state_next, var_state_next,
                  mu_state_filt, var_state_filt,
                  mu_state_pred, var_state_pred,
                  wgt_state)
    if np.isclose(rel_err(mu_state_smooth, mu_state_smooth2), 0.0) and \
        np.isclose(rel_err(var_state_smooth, var_state_smooth2), 0.0):
            return True
    return False

In [49]:
print(test_predict())
print(test_update())
print(test_smooth_mv())

True
True
True


In [ ]:
def ss2gss2(wgt_state, mu_state, var_state, wgt_meas, mu_meas, var_meas):
    n_state = len(mu_state)
    n_meas = len(mu_meas)
    n_gen = n_state + n_meas
    wgt_gen = np.zeros((n_gen, n_gen))
    mu_gen = np.zeros(n_gen)
    var_gen = np.zeros((n_gen, n_gen))
    
    #AA
    wgt_gen[0:n_state, 0:n_state] = wgt_state
    wgt_gen[n_state:, 0:n_state] = wgt_meas.dot(wgt_state)
    
    #BB
    mu_gen[0:n_state] = mu_state
    mu_gen[n_state:] = wgt_meas.dot(mu_state) + mu_meas

    #CC
    Ccn = np.zeros((len_y, len_y))
    Ccn[0:p,0:p] = CC[:,:,i]
    Ccn[p,0:p] = DD[:,i].dot(CC[:,:,i])
    Ccn[p,p] = FF[:,i]
    Cc[:,:,i] = Ccn
        
    Aa[:,:,0] = np.nan
    return Aa, bB, Cc

# MVGSS

To differentiate the notation used in the general model and the state-space model. I will use double letter for the general model. For example, we have

$AA_n =\begin{bmatrix} A_n & 0 \\ D_nA_n & 0 \end{bmatrix}$, 
$bb_n = \begin{bmatrix} b_n \\ D_nb_n + e_n \end{bmatrix}$, and 
$CC_n = \begin{bmatrix} C_n & 0 \\ D_nC_n & F_n \end{bmatrix}$.

We also have 

$bb_0 = \begin{bmatrix} b_0 \\ D_0b_0 \end{bmatrix}$ and 
$CC_0 = \begin{bmatrix} C_0 & 0 \\ D_0C_0 & F_0 \end{bmatrix}$

where 

$A_n = A$, $b_0 = \lambda$, $b_n = \lambda - A \lambda$, $D_n = a_\star'$, $e_n = 0$, $F_0 = 0$ and $F_n^2 = \sigma_n^2 = a_\star'\Sigma_{n|n-1}a_\star$.

In [12]:
C = np.linalg.cholesky(V)
D = np.pad(a, (0, p-q-1), 'constant', constant_values=(0,0)) #a_star

AA = np.dstack([A]*(N+1))
bb = np.repeat(b[:,np.newaxis],N+1,axis=1)
CC = np.dstack([C]*(N+1))
DD = np.repeat(D[:,np.newaxis],N+1,axis=1)
ee = np.zeros((1, N+1))
FF = np.zeros((1, N+1))

AA[:,:,0] = np.nan
bb[:,0] = Y0
V_inf = cov_car([], roots, sigma, v_infinity=True) 
#CC[:,:,0] = np.linalg.cholesky(V_inf)
CC[:,:,0] = np.zeros((CC.shape[0], CC.shape[0]))

for i in range(N+1):
    FF[:,i] = np.sqrt(np.linalg.multi_dot([D.T, Sigma_preds[i], D]))


In [11]:
def ss2gss(AA, bb, CC, DD, ee, FF):
    p = bb.shape[0]
    len_y = bb.shape[0] + ee.shape[0]
    N = ee.shape[1] - 1
    
    Aa = np.zeros((len_y, len_y, N+1))
    bB = np.zeros((len_y, N+1))
    Cc = np.zeros((len_y, len_y, N+1))
    for i in range(N+1):
        #Aa
        Aan = np.zeros((len_y, len_y))
        Aan[0:p,0:p] = AA[:,:,i]
        Aan[p,0:p] = DD[:,i].dot(AA[:,:,i])
        Aa[:,:,i] = Aan
        
        #bB
        bBn = np.zeros(len_y)
        bBn[0:p] = bb[:,i]
        bBn[p] = DD[:,i].dot(bb[:,i]) + ee[:,i]
        bB[:,i]  = bBn
        
        #Cc
        Ccn = np.zeros((len_y, len_y))
        Ccn[0:p,0:p] = CC[:,:,i]
        Ccn[p,0:p] = DD[:,i].dot(CC[:,:,i])
        Ccn[p,p] = FF[:,i]
        Cc[:,:,i] = Ccn
        
    Aa[:,:,0] = np.nan
    return Aa, bB, Cc


In [13]:
def mvGSS(AA, bb, CC):
    d,veclen = bb.shape
    N = veclen-1
    D = d*N + d
    An_m = np.zeros((d,d,N+1,N+1))
    for n in range(N+1):
        for m in range(N+1):
            if m>n:
                An_m[:,:,n,m] = np.eye(d)
            elif n==m:
                An_m[:,:,n,m] = AA[:,:,n]
            else:
                diff = n-m
                A_diff = AA[:,:,m]
                for i in range(diff):
                    A_diff = np.matmul(AA[:,:,m+i+1],A_diff)
                An_m[:,:,n,m] = A_diff
    L = np.zeros((D,D))
    mean_Y = np.zeros(D)
    for n in range(N+1):
        for m in range(n,N+1):
            if n == N:
                L[m*d:m*d+d,n*d:n*d+d] = np.matmul(np.eye(d), CC[:,:,n])
            else:
                L[m*d:m*d+d,n*d:n*d+d] = np.matmul(An_m[:,:,m,n+1], CC[:,:,n])
        for l in range(n):
            mean_Y[n*d:n*d+d] = mean_Y[n*d:n*d+d] + An_m[:,:,n,l+1].dot(bb[:,l])
    LL = L.dot(L.T)
    var_Y = LL
    return An_m, L, mean_Y, var_Y

In [14]:
Aa, bB, Cc = ss2gss(AA,bb,CC,DD,ee,FF)

In [15]:
An_m, L, mean_Y, var_Y = mvGSS(Aa,bB,Cc)

In [16]:
#Check Y_0
var_YY_0 = np.matmul(Cc[:,:,0], Cc[:,:,0].T)
np.allclose(var_Y[0:5, 0:5],var_YY_0)

True

In [17]:
#Check Y_1
var_YY_1 = np.linalg.multi_dot([Aa[:,:,1], var_YY_0, Aa[:,:,1].T]) + np.matmul(Cc[:,:,1],Cc[:,:,1].T)
np.allclose(var_Y[5:10, 5:10],var_YY_1)

True

In [18]:
#Check Y_2
var_YY_2 = np.linalg.multi_dot([Aa[:,:,2], var_YY_1, Aa[:,:,2].T]) + np.matmul(Cc[:,:,2],Cc[:,:,2].T)
np.allclose(var_Y[10:15, 10:15],var_YY_2)

True

### Kalman Filter Sanity Check

In [19]:
#Kalman Filter p(Z_2|X_{0:2})
icond = np.array([False, False, False, False, True]*2 + [False, False, False, False, False]*(N-2))
R,s,T = mvncond(mean_Y[5:], var_Y[5:,5:], icond)

In [20]:
np.allclose(R.dot(us[1:3]).flatten()[4:] + s[4:], mu_currs[2])

True

In [21]:
np.allclose(Sigma_currs[2], T[4:8,4:8])

True

### Kalman Smoothing Sampler

In [22]:
#Kalman Filter p(Z_1|X_{0:2}, Y_2)
icond = np.array([False, False, False, False, True] + [True, True, True, True, True])
R,s,T = mvncond(mean_Y[5:], var_Y[5:,5:], icond)

In [23]:
Smooth_obvs = np.append(np.append(us[1],Y_tt[2]),us[2])

In [24]:
np.allclose(htt[1], R.dot(Smooth_obvs).flatten() + s)

True

In [25]:
np.allclose(Wtt[1],T)

True